In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

data = pd.read_csv(r'airlines_reviews.csv')

features = ['Seat Comfort', 'Staff Service', 'Food & Beverages', 'Inflight Entertainment', 'Value For Money']

airlines_features = data[features]
airlines_names = data['Airline']

user_profiles = {}
for user_id, user_data in data.groupby('Name'):
    user_rated_airline = user_data.iloc[0]  # Get the first airline they rated
    user_profile = user_rated_airline[features].values  # Extract the feature vector
    user_profiles[user_id] = {
        "profile": user_profile,
        "rated_airline": user_rated_airline['Airline']  # Store the airline the user rated
    }

airline_matrix = airlines_features.values  # Convert airline features to a matrix
recommendations = {}

for user_id, user_data in user_profiles.items():
    user_profile = user_data["profile"]
    rated_airline = user_data["rated_airline"]

    user_profile_reshaped = user_profile.reshape(1, -1)
    similarity_scores = cosine_similarity(user_profile_reshaped, airline_matrix)

    similarity_scores = similarity_scores.flatten()
    top_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order

    top_indices = [i for i in top_indices if airlines_names.iloc[i] != rated_airline]

    top_airlines = data['Airline'].iloc[top_indices].unique()[:5]

    recommendations[user_id] = top_airlines

for user_id, recs in list(recommendations.items())[:5]:  # Show for first 5 users
    print(f"Recommendations for User {user_id}: {recs}")

Recommendations for User A Abdel: ['Emirates' 'Cathay Pacific Airways' 'Air France' 'Singapore Airlines'
 'Qatar Airways']
Recommendations for User A Abed: ['Turkish Airlines' 'Korean Air' 'Air France' 'Singapore Airlines'
 'Japan Airlines']
Recommendations for User A Ahmed: ['Singapore Airlines' 'Cathay Pacific Airways' 'All Nippon Airways'
 'Turkish Airlines' 'Emirates']
Recommendations for User A Alhilou: ['Air France' 'Turkish Airlines' 'Korean Air' 'All Nippon Airways'
 'Qatar Airways']
Recommendations for User A Almarani: ['Turkish Airlines' 'Qatar Airways' 'Cathay Pacific Airways'
 'Singapore Airlines' 'Air France']


In [4]:
predicted_ratings = {}

for user_id, recs in recommendations.items():
    user_profile = user_profiles[user_id]["profile"]
    rated_airline = user_profiles[user_id]["rated_airline"]

    predicted_ratings[user_id] = {}

    for airline in recs:
        airline_indices = data.index[data['Airline'] == airline].tolist()

        similarities = cosine_similarity(user_profile.reshape(1, -1), airlines_features.iloc[airline_indices].values)
        similarities = similarities.flatten()

        predicted_ratings_for_airline = {}
        for idx, feature in enumerate(features):
            weighted_ratings = airlines_features.iloc[airline_indices][feature].values * similarities
            predicted_rating = weighted_ratings.sum() / similarities.sum()  # Weighted average
            predicted_ratings_for_airline[feature] = predicted_rating

        predicted_ratings[user_id][airline] = predicted_ratings_for_airline

for user_id, recs in list(predicted_ratings.items())[:5]:  # Show for first 5 users
    print(f"\nPredicted Ratings for User {user_id}:")
    for airline, features_ratings in recs.items():
        print(f"  Airline: {airline}")
        for feature, rating in features_ratings.items():
            print(f"    {feature}: {rating:.1f}")


Predicted Ratings for User A Abdel:
  Airline: Emirates
    Seat Comfort: 3.2
    Staff Service: 3.1
    Food & Beverages: 3.0
    Inflight Entertainment: 3.8
    Value For Money: 2.7
  Airline: Cathay Pacific Airways
    Seat Comfort: 3.6
    Staff Service: 3.8
    Food & Beverages: 3.2
    Inflight Entertainment: 3.9
    Value For Money: 3.3
  Airline: Air France
    Seat Comfort: 2.9
    Staff Service: 3.5
    Food & Beverages: 3.0
    Inflight Entertainment: 3.2
    Value For Money: 2.6
  Airline: Singapore Airlines
    Seat Comfort: 3.7
    Staff Service: 4.1
    Food & Beverages: 3.5
    Inflight Entertainment: 3.9
    Value For Money: 3.4
  Airline: Qatar Airways
    Seat Comfort: 4.0
    Staff Service: 4.4
    Food & Beverages: 3.9
    Inflight Entertainment: 4.2
    Value For Money: 3.8

Predicted Ratings for User A Abed:
  Airline: Turkish Airlines
    Seat Comfort: 2.7
    Staff Service: 3.1
    Food & Beverages: 3.0
    Inflight Entertainment: 3.0
    Value For Money: 2.5


In [34]:
#Top 5 recommendation
import tkinter as tk
from tkinter import messagebox

def search_user():
    user_name = entry.get()
    if user_name in recommendations:
        results_frame.pack_forget()
        for widget in results_frame.winfo_children():
            widget.destroy()
        
        tk.Label(results_frame, text=f"Top 5 Recommendations for {user_name}:", font=("Arial", 14, "bold"), fg="#003366", bg="#E6F7FF").pack(pady=10)

        for airline in recommendations[user_name]:
            airline_button = tk.Button(results_frame, text=airline, font=("Arial", 12, "bold"), fg="#003366", bg="#B3E5FC", activebackground="#81D4FA",
                                       command=lambda a=airline: show_predicted_ratings(user_name, a))
            airline_button.pack(pady=5, padx=10, fill="x")
        
        results_frame.pack(fill="both", expand=True, padx=10, pady=10)
    else:
        messagebox.showerror("Error", "User not found in recommendations.")

def show_predicted_ratings(user_name, airline):
    if airline in predicted_ratings[user_name]:
        predicted = predicted_ratings[user_name][airline]
        ratings_text = f"Predicted Ratings for {airline}:\n" + "\n".join([f"{feature}: {rating:.1f}" for feature, rating in predicted.items()])
        messagebox.showinfo(f"{airline} Ratings", ratings_text)
    else:
        messagebox.showerror("Error", f"No predicted ratings available for {airline}.")

root = tk.Tk()
root.title("Airline Recommendation System")
root.geometry("450x500")
root.configure(bg="#E6F7FF")  

frame = tk.Frame(root, bg="#E6F7FF")
frame.pack(pady=20)

tk.Label(frame, text="Enter User Name:", font=("Arial", 12, "bold"), fg="#003366", bg="#E6F7FF").grid(row=0, column=0, padx=5)

entry = tk.Entry(frame, width=20, font=("Arial", 12))
entry.grid(row=0, column=1, padx=5)

search_button = tk.Button(frame, text="Search", font=("Arial", 12, "bold"), fg="white", bg="#007FFF", activebackground="#0052CC", command=search_user)
search_button.grid(row=0, column=2, padx=5)

results_frame = tk.Frame(root, bg="#E6F7FF")

root.mainloop()

In [35]:
#New user recommendation

import tkinter as tk
from tkinter import messagebox
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

airlines_df = data

features = ["Seat Comfort", "Staff Service", "Food & Beverages", "Inflight Entertainment", "Value For Money"]

def recommend_airline(new_user_ratings):
    user_profile = np.array(new_user_ratings).reshape(1, -1)
    airline_features = airlines_df[features].values
    similarities = cosine_similarity(user_profile, airline_features).flatten()
    best_match_index = np.argmax(similarities)
    recommended_airline = airlines_df.iloc[best_match_index]["Airline"]
    return recommended_airline

def submit_new_user():
    try:
        name = entry_name.get()
        seat_comfort = float(entry_seat_comfort.get())
        staff_service = float(entry_staff_service.get())
        food_beverages = float(entry_food_beverages.get())
        inflight_entertainment = float(entry_inflight_entertainment.get())
        value_for_money = float(entry_value_for_money.get())

        new_user_ratings = [seat_comfort, staff_service, food_beverages, inflight_entertainment, value_for_money]
        recommended_airline = recommend_airline(new_user_ratings)
        
        messagebox.showinfo("Recommendation", f"Recommended Airline for {name}: {recommended_airline}")
    
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numerical ratings for all factors.")

root = tk.Tk()
root.title("New User Airline Recommendation")
root.geometry("400x400")
root.configure(bg="#E6F7FF") 

tk.Label(root, text="Enter Your Name:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=0, column=0, padx=10, pady=5, sticky="w")
entry_name = tk.Entry(root, font=("Arial", 12))
entry_name.grid(row=0, column=1, padx=10, pady=5)

tk.Label(root, text="Seat Comfort:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=1, column=0, padx=10, pady=5, sticky="w")
entry_seat_comfort = tk.Entry(root, font=("Arial", 12))
entry_seat_comfort.grid(row=1, column=1, padx=10, pady=5)

tk.Label(root, text="Staff Service:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=2, column=0, padx=10, pady=5, sticky="w")
entry_staff_service = tk.Entry(root, font=("Arial", 12))
entry_staff_service.grid(row=2, column=1, padx=10, pady=5)

tk.Label(root, text="Food & Beverages:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=3, column=0, padx=10, pady=5, sticky="w")
entry_food_beverages = tk.Entry(root, font=("Arial", 12))
entry_food_beverages.grid(row=3, column=1, padx=10, pady=5)

tk.Label(root, text="Inflight Entertainment:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=4, column=0, padx=10, pady=5, sticky="w")
entry_inflight_entertainment = tk.Entry(root, font=("Arial", 12))
entry_inflight_entertainment.grid(row=4, column=1, padx=10, pady=5)

tk.Label(root, text="Value For Money:", font=("Arial", 12, "bold"), bg="#E6F7FF", fg="#003366").grid(row=5, column=0, padx=10, pady=5, sticky="w")
entry_value_for_money = tk.Entry(root, font=("Arial", 12))
entry_value_for_money.grid(row=5, column=1, padx=10, pady=5)

submit_button = tk.Button(root, text="Submit", font=("Arial", 12, "bold"), fg="white", bg="#007FFF", activebackground="#0052CC", command=submit_new_user)
submit_button.grid(row=6, column=0, columnspan=2, pady=20)

root.mainloop()